In [1]:
import tensorflow as tf 
number_gpu = 2 
tf.config.experimental.set_visible_devices(
    tf.config.experimental.list_physical_devices('GPU')[number_gpu],
    'GPU'
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.models import Sequential

import configparser
import os 
from pathlib import Path
import pandas as pd
import json
import re
import numpy as np

2025-04-04 19:56:12.844412: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-04 19:56:12.993795: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 19:56:13.697189: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-04-04 19:56:13.697249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
base_dir = Path(os.getcwd()).resolve().parent
config = configparser.ConfigParser()
config.read(os.path.join(base_dir,'config.ini'))

['/home/lamedinaa/testing_rl/config.ini']

In [3]:
autor = 'nihal'
str_ftime = config.get(autor,'strftime_data_file')
name_file_abstract_datasets = f"abstract_episodes_trnng_{autor}_{str_ftime}.csv"
path_abstract_datasets_dir = config.get(autor,'path_abstract_datasets')
path_file_abstract_datasets = os.path.join(base_dir,path_abstract_datasets_dir,name_file_abstract_datasets)
print(path_file_abstract_datasets)

name_file_abstract_states = f'abstract_states_trnng_{autor}_{str_ftime}.json'
path_abstract_states_dir = config.get(autor,'path_abstract_states_dir')
path_file_abstract_states = os.path.join(base_dir,path_abstract_states_dir,name_file_abstract_states)
print(path_file_abstract_states)

f = open(path_file_abstract_states,'r')
dict_abstract_states = json.load(f)

df_abstract_states = pd.read_csv(path_file_abstract_datasets,sep=';')

/home/lamedinaa/testing_rl/data/4_random_forest_model/datasets/abstract_episodes_trnng_nihal_351905032025.csv
/home/lamedinaa/testing_rl/data/2_abstract_classes/abstract_states_trnng_nihal_351905032025.json


In [4]:
tokenizer = Tokenizer()
actions_set = ["1","0"]
vocabulary = list(dict_abstract_states.keys()) + actions_set

tokenizer.word_index = {
    f'w{i+1}' if class_abstract in dict_abstract_states.keys() else class_abstract : i + 1 for i,class_abstract in enumerate(vocabulary)}

### EJEMPLOR TOKENIZER
text = [df_abstract_states['parse_abstract_states'][389]]
sequence = tokenizer.texts_to_sequences(text)
print(text)
print(sequence)

['w33 1 w33 1 w6 0 w6 0 w6 1 w6 0 w32 1 w32 0 w32 0 w32 1 w17 0 w17 1 w17 0 w17 1 w1 0 w1 0 w1 1 w1 0 w20 1 w20 0 w20 1 w20 0 w12 1 w12 0 w12 1 w12 0 w10 1 w10 0 w10 0 w10 1 w4 0 w4 1 w4 0 w4 1 w26 0 w26 1 w26 0 w26 1 w15 0 w15 1 w15 0 w15 1 w2 0 w2 0 w2 1 [[-0.20469867 -0.52386    -0.09189483 -0.14490547]] 0 w3 1 w3 0 w3 1 w3 0 w23 1 w23 0 w23 1 w23 0 w35 0 w35 1 w35 0 w35 1 w11 0 w11 1 w11 0 w11 0 w28 1 w28 0 w28 1 w28 0 w8 0 w8 1 w8 0 w8 1 w21 0 w21 1 w21 0 w21 0 w34 1 w34 0 w34 1 w34 0 w31 1 w31 0 w31 0 w31 1 w29 0 w29 1 w29 0 w29 0 w9 1 w9 0 w9 1 w9 0 w19 0 w19 1 w19 0 w19 1 w13 0 w13 1 w13 0 w13 0 w18 1 w18 0 w18 1 w18 0 w7 1 w7 0 w7 1 w7 0 w5 0 w5 1 w5 0 w5 1 w16 0 w16 1 w16 0']
[[33, 52, 33, 52, 6, 53, 6, 53, 6, 52, 6, 53, 32, 52, 32, 53, 32, 53, 32, 52, 17, 53, 17, 52, 17, 53, 17, 52, 1, 53, 1, 53, 1, 52, 1, 53, 20, 52, 20, 53, 20, 52, 20, 53, 12, 52, 12, 53, 12, 52, 12, 53, 10, 52, 10, 53, 10, 53, 10, 52, 4, 53, 4, 52, 4, 53, 4, 52, 26, 53, 26, 52, 26, 53, 26, 52, 15, 53, 15,

In [5]:
print(len(vocabulary),len(tokenizer.word_index)+1)

53 54


In [6]:
input_sequences =  []
for index,row in df_abstract_states.iterrows():
    token_list = tokenizer.texts_to_sequences([row['parse_abstract_states']])[0]
    
    for i in range(1,len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

In [7]:
max_sequence_length = max([len(sequence) for sequence in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, 
    maxlen=max_sequence_length, 
    padding='pre'
))

In [8]:
predictors = input_sequences[:,:-1]
labels = input_sequences[:,-1]
total_words = len(tokenizer.word_index) + 1
labels =  to_categorical(labels,num_classes=total_words)
input_len = max_sequence_length -1 

In [9]:
df_abstract_states.shape

(390, 7)

# Model LSTM

In [10]:
total_words

54

In [11]:
model = Sequential()
model.add(Embedding(total_words,50,input_length=input_len))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(total_words,activation='softmax'))
model.summary()

2025-04-04 19:57:29.939537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-04 19:57:30.610168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43103 MB memory:  -> device: 2, name: NVIDIA A40, pci bus id: 0000:ca:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 408, 50)           2700      
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 54)                5454      
                                                                 
Total params: 68,554
Trainable params: 68,554
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [13]:
model.fit(predictors,labels,epochs=5,verbose=1)

Epoch 1/5


2025-04-04 19:57:33.762644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2025-04-04 19:57:34.516533: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-04-04 19:57:34.518613: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7ef7016fa180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-04 19:57:34.518631: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2025-04-04 19:57:34.523568: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-04 19:57:34.599438: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-04-04 19:57:34.666541: I tensorfl

2541/2541 [==============================] - 98s 37ms/step - loss: 0.8351
Epoch 2/5
2541/2541 [==============================] - 90s 36ms/step - loss: 0.3361
Epoch 3/5
2541/2541 [==============================] - 90s 35ms/step - loss: 0.2904
Epoch 4/5
2541/2541 [==============================] - 91s 36ms/step - loss: 0.2711
Epoch 5/5
2541/2541 [==============================] - 90s 36ms/step - loss: 0.2528


In [14]:
model.save('lstm_rl')

INFO:tensorflow:Assets written to: lstm_rl/assets


INFO:tensorflow:Assets written to: lstm_rl/assets


# MODELO

In [1]:
import tensorflow as tf
number_gpu = 2 
tf.config.experimental.set_visible_devices(
    tf.config.experimental.list_physical_devices('GPU')[number_gpu],
    'GPU',
)


2025-04-03 19:00:42.089699: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-03 19:00:42.235721: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 19:00:42.964852: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-04-03 19:00:42.964908: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
from tensorflow.keras.models import load_model
charge_model = load_model('lstm_rl')

In [3]:
charge_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 408, 50)           2700      
                                                                 
 lstm_1 (LSTM)               (None, 100)               60400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 54)                5454      
                                                                 
Total params: 68,554
Trainable params: 68,554
Non-trainable params: 0
_________________________________________________________________


In [10]:
input_shape_model = charge_model.layers[0].input_shape[1]

print(input_shape_model)

408


In [49]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import configparser
import os
from pathlib import Path
import json
import numpy as np

In [34]:
base_dir = Path(os.getcwd()).resolve().parent
config = configparser.ConfigParser()
config.read(os.path.join(base_dir,'config.ini'))

autor = 'nihal'
str_ftime = config.get(autor,'strftime_data_file')
name_file_abstract_states = f'abstract_states_trnng_{autor}_{str_ftime}.json'
path_abstract_states_dir = config.get(autor,'path_abstract_states_dir')
path_file_abstract_states = os.path.join(base_dir,path_abstract_states_dir,name_file_abstract_states)
print(path_file_abstract_states)
f = open(path_file_abstract_states,'r')
dict_abstract_states = json.load(f)

tokenizer = Tokenizer()
actions_set = ["0","1"]
vocabulary = list(dict_abstract_states.keys()) + actions_set
tokenizer.word_index = {
    f'w{i+1}' if class_abstract in dict_abstract_states.keys() else class_abstract : i + 1 for i,class_abstract in enumerate(vocabulary)}
tokenizer.index_word = {value:key for key,value in tokenizer.word_index.items()}

/home/lamedinaa/testing_rl/data/2_abstract_classes/abstract_states_trnng_nihal_351905032025.json


In [78]:
def predict_next_token(seed_text): 
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list],input_shape_model,padding='pre')
    prediction_list = charge_model.predict(token_list)
    prediction = np.argmax(prediction_list,axis=1)
    print(f'prediction: {prediction[0]}')
    print(f'Probability: {prediction_list[0][prediction[0]]}')
    return prediction,prediction_list

In [121]:
prediction,prediction_list = predict_next_token('w32 0 w32 0 w32 0 w32')
prediction

1/1 [==============================] - 0s 36ms/step
prediction: 53
Probability: 0.966880202293396


array([53])

In [126]:
def generate_headline(seed_text,next_words=20): 
    for _ in range(next_words): 
        prediction,_ = predict_next_token(seed_text)
        next_word = tokenizer.sequences_to_texts([prediction])[0]
        seed_text += ' '+ next_word 
        print(seed_text)
    return seed_text

In [127]:
generate_headline('w32',next_words=50)


1/1 [==============================] - 0s 43ms/step
prediction: 52
Probability: 0.9336005449295044
w32 0
1/1 [==============================] - 0s 27ms/step
prediction: 32
Probability: 0.8086513876914978
w32 0 w32
1/1 [==============================] - 0s 26ms/step
prediction: 52
Probability: 0.8011139631271362
w32 0 w32 0
1/1 [==============================] - 0s 29ms/step
prediction: 32
Probability: 0.8328719735145569
w32 0 w32 0 w32
1/1 [==============================] - 0s 28ms/step
prediction: 53
Probability: 0.8664010763168335
w32 0 w32 0 w32 1
1/1 [==============================] - 0s 28ms/step
prediction: 32
Probability: 0.6403516530990601
w32 0 w32 0 w32 1 w32
1/1 [==============================] - 0s 28ms/step
prediction: 52
Probability: 0.5392526984214783
w32 0 w32 0 w32 1 w32 0
1/1 [==============================] - 0s 28ms/step
prediction: 17
Probability: 0.6491907835006714
w32 0 w32 0 w32 1 w32 0 w17
1/1 [==============================] - 0s 27ms/step
prediction: 53
Proba

'w32 0 w32 0 w32 1 w32 0 w17 1 w17 1 w17 0 w17 1 w1 0 w1 1 w1 0 w1 1 w20 0 w20 1 w20 0 w20 1 w12 0 w12 1 w12 0 w12 1 w10 0 w10 1 w10 0 w10 1 w4 0 w4'

In [128]:
generate_headline('w27',next_words=50)

1/1 [==============================] - 0s 33ms/step
prediction: 52
Probability: 0.9232657551765442
w27 0
1/1 [==============================] - 0s 25ms/step
prediction: 27
Probability: 0.7535117268562317
w27 0 w27
1/1 [==============================] - 0s 25ms/step
prediction: 52
Probability: 0.8767423629760742
w27 0 w27 0
1/1 [==============================] - 0s 27ms/step
prediction: 27
Probability: 0.6493148803710938
w27 0 w27 0 w27
1/1 [==============================] - 0s 26ms/step
prediction: 52
Probability: 0.5330126285552979
w27 0 w27 0 w27 0
1/1 [==============================] - 0s 26ms/step
prediction: 22
Probability: 0.6317209005355835
w27 0 w27 0 w27 0 w22
1/1 [==============================] - 0s 33ms/step
prediction: 53
Probability: 0.8495853543281555
w27 0 w27 0 w27 0 w22 1
1/1 [==============================] - 0s 25ms/step
prediction: 22
Probability: 0.9936874508857727
w27 0 w27 0 w27 0 w22 1 w22
1/1 [==============================] - 0s 25ms/step
prediction: 53
Proba

'w27 0 w27 0 w27 0 w22 1 w22 1 w22 0 w22 1 w25 0 w25 1 w25 1 w25 0 w30 1 w30 0 w30 1 w30 0 w33 1 w33 0 w33 1 w33 0 w6 1 w6 0 w6 1 w6 0 w32 1 w32 0 w32'